# [모의 캐글-의료] 흉부 CT 코로나 감염 여부 분류
- 이미지 binary 분류 과제
- 담당: 이녕민M

## Import Libraries

In [ ]:
# !apt-get update && apt-get install -y python3-opencv

In [519]:
# !pip install sklearn
# ! pip install mxnet
# !pip install albumentations

In [2]:
! pip install mxnet

     |████████████████████████████████| 47.3 MB 23.9 MB/s eta 0:00:01


In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torchvision.utils
from sklearn.model_selection import KFold

# import mxnet as mx
import random
import albumentations
from albumentations.pytorch import ToTensorV2

## Set Arguments & hyperparameters

In [2]:
# 시드(seed) 설정

RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [3]:
# parameters

### 데이터 디렉토리 설정 ###
DATA_DIR= 'data'
NUM_CLS = 2

EPOCHS = 50
BATCH_SIZE = 16
LEARNING_RATE = 0.0001
EARLY_STOPPING_PATIENCE = 15
INPUT_SHAPE = 256 # 크게하는게 좋다고 함

k = 5

os.environ["CUDA_VISIBLE_DEVICES"]="0"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

#### Train & Validation Set loader

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, mode, input_shape):
        self.data_dir = data_dir
        self.mode = mode
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        if self.mode == 'train':
            self.db = self.db[:int(len(self.db) * 0.8)]
        elif self.mode == 'val':
            self.db = self.db[int(len(self.db) * 0.8):]
            self.db.reset_index(inplace=True)
        else:
            print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        # self.transform = transforms.Compose([transforms.Resize(self.input_shape),
        #                                      transforms.RandomHorizontalFlip(p=0.5),
        #                                      transforms.ToTensor(),
        #                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.transform = albumentations.Compose([albumentations.Resize(256, 256), 
                                                 albumentations.RandomCrop(240, 240),
                                                 albumentations.HorizontalFlip(), # Same with transforms.RandomHorizontalFlip()
                                                 albumentations.VerticalFlip(),
                                                 albumentations.ChannelShuffle(0.05),
                                                 albumentations.RandomBrightnessContrast(p=0.5),
                                                albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                albumentations.pytorch.transforms.ToTensorV2()])
        
        
    def data_loader(self):
        print('Loading ' + self.mode + ' dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        image = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB)
        trans_image = self.transform(image = image)
        trans_image = trans_image['image']
        
        
        # trans_image = self.transform(Image.fromarray(cvimg))
        

        return trans_image, data['COVID']

## Model

In [8]:
# import torch.nn.functional as F

# class custom_CNN(nn.Module):
#     def __init__(self, num_classes):
#         super(custom_CNN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5)
#         self.pool = nn.MaxPool2d(kernel_size=2)
#         self.conv2 = nn.Conv2d(in_channels=8, out_channels=25, kernel_size=5)
        
#         self.fc1 = nn.Linear(in_features=25*29*29, out_features=128)
#         self.fc2 = nn.Linear(in_features=128, out_features=num_classes)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x))) # (32, 3, 128, 128) -> (32, 8, 62, 62)
#         x = self.pool(F.relu(self.conv2(x))) # (32, 8, 62, 62) -> (32, 25, 29, 29)
        
#         x = torch.flatten(x,1)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
        
#         output = self.softmax(x)
        
#         return output

In [5]:
import torchvision.models as models

class resnet(nn.Module):
    def __init__(self, numclasses):
        super(resnet, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, numclasses)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, input_img):
        x = self.model(input_img)
        x = self.softmax(x)
        return x

In [6]:
# import torchvision.models as models

# class vgg16(nn.Module):
#     def __init__(self, numclasses):
#         super(vgg16, self).__init__()
#         self.model = models.vgg16(pretrained=False)
#         self.model.classifier[6] = nn.Linear(in_features=4096, out_features=numclasses, bias=True)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, input_img):
#         x = self.model(input_img)
#         x = self.softmax(x)
#         return x

In [5]:
# import torchvision.models as models

# class resnet152(nn.Module):
#     def __init__(self, numclasses):
#         super(resnet152, self).__init__()
#         self.model = models.resnet152(pretrained=False)
#         self.model.fc = nn.Linear(self.model.fc.in_features, numclasses)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, input_img):
#         x = self.model(input_img)
#         x = self.softmax(x)
#         return x

In [14]:
# import torchvision.models as models

# class regnet_6gf(nn.Module):
#     def __init__(self, numclasses):
#         super(regnet_6gf, self).__init__()
#         self.model = models.regnet_y_1_6gf(pretrained=False)
#         self.model.fc = nn.Linear(self.model.fc.in_features, numclasses)
#         self.softmax = nn.Softmax(dim=1)
        
#     def forward(self, input_img):
#         x = self.model(input_img)
#         x = self.softmax(x)
#         return x

## Utils
### EarlyStopper

In [6]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단

    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)

### Trainer

In [7]:
class Trainer():
    """ epoch에 대한 학습 및 검증 절차 정의"""
    
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        """ 초기화
        """
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.metric_fn = metric_fn

    def train_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 학습 절차"""
        
        self.model.train()
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Train loss: {self.train_mean_loss}, Acc: {self.train_score}, F1-Macro: {f1}'
        print(msg)

    def validate_epoch(self, dataloader, epoch_index):
        """ 한 epoch에서 수행되는 검증 절차
        """
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            pred = self.model(img)
            
            loss = self.loss_fn(pred[:,1], label)
            val_total_loss += loss.item()
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        msg = f'Epoch {epoch_index}, Val loss: {self.val_mean_loss}, Acc: {self.validation_score}, F1-Macro: {f1}'
        print(msg)

### Metrics

In [8]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

## Train
### 학습을 위한 객체 선언

#### Load Dataset & Dataloader

In [9]:
# Load dataset & dataloader
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 516 Val set samples: 130


In [9]:
# Load dataset & dataloader (ab)
train_dataset = CustomDataset(data_dir=DATA_DIR, mode='train', input_shape=INPUT_SHAPE)
validation_dataset = CustomDataset(data_dir=DATA_DIR, mode='val', input_shape=INPUT_SHAPE)

dataset = ConcatDataset([train_dataset, validation_dataset])
splits=KFold(n_splits=k,shuffle=True, random_state=42)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True)
print('Train set samples:',len(train_dataset),  'Val set samples:', len(validation_dataset))

Loading train dataset..
Loading val dataset..
Train set samples: 516 Val set samples: 130


#### Load model and other utils

In [10]:
# Load Model
# model = custom_CNN(NUM_CLS).to(DEVICE)
model = resnet(NUM_CLS).to(DEVICE)

# # Save Initial Model
# torch.save(model.state_dict(), 'initial.pt')

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(train_dataloader))
loss_fn = nn.BCELoss()
metric_fn = get_metric_fn

# Set trainer
trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)

# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)

In [11]:
import gc
gc.collect()
torch.cuda.empty_cache()

### epoch 단위 학습 진행

In [12]:
for epoch_index in tqdm(range(EPOCHS)):

    trainer.train_epoch(train_dataloader, epoch_index)
    trainer.validate_epoch(validation_dataloader, epoch_index)

    # early_stopping check
    early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

    if early_stopper.stop:
        print('Early stopped')
        break

    if early_stopper.save_model:
        check_point = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(check_point, 'best.pt')

  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7261331677436829, Acc: 0.5852713178294574, F1-Macro: 0.580328367284889


  2% 1/50 [00:07<06:30,  7.96s/it]

Epoch 0, Val loss: 0.7711303010582924, Acc: 0.6076923076923076, F1-Macro: 0.5770334928229665
Epoch 1, Train loss: 0.6551164649426937, Acc: 0.6492248062015504, F1-Macro: 0.6340726635871035
Epoch 1, Val loss: 0.7365354672074318, Acc: 0.6076923076923076, F1-Macro: 0.6037296037296038
Validation loss decreased 0.7711303010582924 -> 0.7365354672074318


  4% 2/50 [00:16<06:25,  8.04s/it]

...

Epoch 20, Train loss: 0.5271640010178089, Acc: 0.7713178294573644, F1-Macro: 0.7691502752544017
Epoch 20, Val loss: 0.9309655055403709, Acc: 0.5615384615384615, F1-Macro: 0.47413242495209706
Early stopping counter 14/15


 42% 21/50 [03:30<05:01, 10.38s/it]

Epoch 21, Train loss: 0.5498710423707962, Acc: 0.7383720930232558, F1-Macro: 0.7349062886499858


 42% 21/50 [03:39<05:02, 10.44s/it]

Epoch 21, Val loss: 0.7178863473236561, Acc: 0.7153846153846154, F1-Macro: 0.714962962962963
Early stopping counter 15/15
Early stopped


In [12]:
for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(dataset)))):

    print('Fold {}'.format(fold + 1))

    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(val_idx)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
    test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=test_sampler)
    

    for epoch_index in tqdm(range(EPOCHS)):

        trainer.train_epoch(train_dataloader, epoch_index)
        trainer.validate_epoch(validation_dataloader, epoch_index)

        # early_stopping check
        early_stopper.check_early_stopping(loss=trainer.val_mean_loss)

        if early_stopper.stop:
            print('Early stopped')
            break

        if early_stopper.save_model:
            check_point = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()
            }
            torch.save(check_point, 'best2.pt')


  0% 0/50 [00:00<?, ?it/s]

Fold 1
Epoch 0, Train loss: 0.7261331677436829, Acc: 0.5852713178294574, F1-Macro: 0.580328367284889


  2% 1/50 [00:08<06:45,  8.28s/it]

Epoch 0, Val loss: 0.7711303010582924, Acc: 0.6076923076923076, F1-Macro: 0.5770334928229665
Epoch 1, Train loss: 0.6551164649426937, Acc: 0.6492248062015504, F1-Macro: 0.6340726635871035
Epoch 1, Val loss: 0.7365354672074318, Acc: 0.6076923076923076, F1-Macro: 0.6037296037296038
Validation loss decreased 0.7711303010582924 -> 0.7365354672074318


  4% 2/50 [00:16<06:39,  8.32s/it]

...

Epoch 0, Val loss: 0.8852848708629608, Acc: 0.6461538461538462, F1-Macro: 0.6201727642276422
Early stopping counter 17/15
Early stopped
Fold 4
Epoch 0, Train loss: 0.523485753685236, Acc: 0.7577519379844961, F1-Macro: 0.7540861727801988


  0% 0/50 [00:08<?, ?it/s]
  0% 0/50 [00:00<?, ?it/s]

Epoch 0, Val loss: 0.7241522297263145, Acc: 0.6384615384615384, F1-Macro: 0.6267790605338709
Early stopping counter 18/15
Early stopped
Fold 5
Epoch 0, Train loss: 0.4861377524212003, Acc: 0.7829457364341085, F1-Macro: 0.7828935505199255


  0% 0/50 [00:08<?, ?it/s]

Epoch 0, Val loss: 0.6216891596559435, Acc: 0.7153846153846154, F1-Macro: 0.715232964300515
Early stopping counter 19/15
Early stopped


## Inference
### 모델 로드

In [13]:
TRAINED_MODEL_PATH = 'best.pt'

### Load dataset

In [14]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Transform function
        # self.transform = transforms.Compose([transforms.Resize(self.input_shape),
        #                                      transforms.ToTensor(),
        #                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        self.transform = albumentations.Compose([albumentations.Resize(256, 256), 
                                                 albumentations.RandomCrop(240, 240),
                                                 albumentations.HorizontalFlip(), # Same with transforms.RandomHorizontalFlip()
                                                 albumentations.VerticalFlip(),
                                                 albumentations.ChannelShuffle(0.05),
                                                 albumentations.RandomBrightnessContrast(p=0.5),
                                                albumentations.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                                albumentations.pytorch.transforms.ToTensorV2()])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        

        # Preprocessing images
        image = cv2.cvtColor(cvimg, cv2.COLOR_BGR2RGB)
        trans_image = self.transform(image = image)
        trans_image = trans_image['image']
        
        
        # trans_image = self.transform(Image.fromarray(cvimg))
        

        # trans_image = self.transform(Image.fromarray(cvimg))

        
        
        
        return trans_image, data['file_name']

In [15]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Loading test dataset..


### 추론 진행

In [16]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

<All keys matched successfully>

In [17]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        print(pred)
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

2it [00:00,  6.57it/s]

tensor([[0.9757, 0.0243],
        [0.9431, 0.0569],
        [0.8767, 0.1233],
        [0.9714, 0.0286],
        [0.9574, 0.0426],
        [0.3201, 0.6799],
        [0.7686, 0.2314],
        [0.9028, 0.0972],
        [0.9660, 0.0340],
        [0.3462, 0.6538],
        [0.9215, 0.0785],
        [0.9545, 0.0455],
        [0.9298, 0.0702],
        [0.9682, 0.0318],
        [0.9835, 0.0165],
        [0.9090, 0.0910]], device='cuda:0')
tensor([[0.9303, 0.0697],
        [0.9533, 0.0467],
        [0.9395, 0.0605],
        [0.8969, 0.1031],
        [0.9768, 0.0232],
        [0.8742, 0.1258],
        [0.9290, 0.0710],
        [0.8877, 0.1123],
        [0.6265, 0.3735],
        [0.9659, 0.0341],
        [0.9648, 0.0352],
        [0.9414, 0.0586],
        [0.1227, 0.8773],
        [0.9291, 0.0709],
        [0.9269, 0.0731],
        [0.5304, 0.4696]], device='cuda:0')


4it [00:00,  6.59it/s]

tensor([[0.5432, 0.4568],
        [0.9781, 0.0219],
        [0.9077, 0.0923],
        [0.8528, 0.1472],
        [0.9430, 0.0570],
        [0.7342, 0.2658],
        [0.4813, 0.5187],
        [0.8800, 0.1200],
        [0.7283, 0.2717],
        [0.8763, 0.1237],
        [0.7490, 0.2510],
        [0.5749, 0.4251],
        [0.8896, 0.1104],
        [0.7538, 0.2462],
        [0.9098, 0.0902],
        [0.9658, 0.0342]], device='cuda:0')
tensor([[0.9526, 0.0474],
        [0.9406, 0.0594],
        [0.9205, 0.0795],
        [0.7294, 0.2706],
        [0.9293, 0.0707],
        [0.7726, 0.2274],
        [0.7726, 0.2274],
        [0.9563, 0.0437],
        [0.8631, 0.1369],
        [0.8501, 0.1499],
        [0.9328, 0.0672],
        [0.7410, 0.2590],
        [0.5924, 0.4076],
        [0.9736, 0.0264],
        [0.9641, 0.0359],
        [0.9007, 0.0993]], device='cuda:0')


6it [00:00,  6.36it/s]

tensor([[0.6243, 0.3757],
        [0.5096, 0.4904],
        [0.3725, 0.6275],
        [0.9044, 0.0956],
        [0.7050, 0.2950],
        [0.7875, 0.2125],
        [0.9264, 0.0736],
        [0.9811, 0.0189],
        [0.7783, 0.2217],
        [0.9311, 0.0689],
        [0.5832, 0.4168],
        [0.8889, 0.1111],
        [0.9913, 0.0087],
        [0.9727, 0.0273],
        [0.8295, 0.1705],
        [0.9150, 0.0850]], device='cuda:0')
tensor([[0.5453, 0.4547],
        [0.8736, 0.1264],
        [0.9181, 0.0819],
        [0.3679, 0.6321],
        [0.6930, 0.3070],
        [0.7859, 0.2141],
        [0.8787, 0.1213],
        [0.9458, 0.0542],
        [0.9524, 0.0476],
        [0.8641, 0.1359],
        [0.8767, 0.1233],
        [0.9788, 0.0212],
        [0.9405, 0.0595],
        [0.9004, 0.0996],
        [0.9907, 0.0093],
        [0.8588, 0.1412]], device='cuda:0')


7it [00:01,  6.76it/s]

tensor([[0.7176, 0.2824],
        [0.8504, 0.1496],
        [0.5633, 0.4367],
        [0.6125, 0.3875]], device='cuda:0')


### 결과 저장

In [18]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('prediction.csv', index=False)